In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
%cd '/content/gdrive/My Drive/Colab Notebooks/Model'

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-4a78438467fa>", line 1, in <cell line: 1>
    get_ipython().run_line_magic('cd', "'/content/gdrive/My Drive/Colab Notebooks/Model'")
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2418, in run_line_magic
    result = fn(*args, **kwargs)
  File "<decorator-gen-85>", line 2, in cd
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/magic.py", line 187, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py", line 342, in cd
    oldcwd = os.getcwd()
OSError: [Errno 107] Transport endpoint is not connected

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packa

In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNetV2
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam

In [3]:
# Data Preparation
train_data_dir = 'MyDataset/train'
validation_data_dir = 'MyDataset/val'
img_height, img_width = 224, 224
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Model Architecture
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(9, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [4]:

# Freeze layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])



In [5]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_2[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[0][0]']      

In [ ]:
# Model Training
epochs = 10
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
)

Epoch 1/10
14/86 [===>..........................] - ETA: 12:13 - loss: 0.6119 - accuracy: 0.7969

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


86/86 [==============================] - 839s 10s/step - loss: 0.2843 - accuracy: 0.9142 - val_loss: 0.1417 - val_accuracy: 0.9375
Epoch 2/10
86/86 [==============================] - 196s 2s/step - loss: 0.1163 - accuracy: 0.9614 - val_loss: 0.1576 - val_accuracy: 0.9453
Epoch 3/10
86/86 [==============================] - 189s 2s/step - loss: 0.0944 - accuracy: 0.9698 - val_loss: 0.1780 - val_accuracy: 0.9453
Epoch 4/10
86/86 [==============================] - 189s 2s/step - loss: 0.0766 - accuracy: 0.9796 - val_loss: 0.2091 - val_accuracy: 0.9375
Epoch 5/10
86/86 [==============================] - 184s 2s/step - loss: 0.0757 - accuracy: 0.9720 - val_loss: 0.2490 - val_accuracy: 0.9453
Epoch 6/10
86/86 [==============================] - 189s 2s/step - loss: 0.0692 - accuracy: 0.9756 - val_loss: 0.2321 - val_accuracy: 0.9453
Epoch 7/10
86/86 [==============================] - 191s 2s/step - loss: 0.0459 - accuracy: 0.9847 - val_loss: 0.2304 - val_accuracy: 0.9375
Epoch 8/10
86/86 [=====

In [ ]:

# Save the trained model
model.save('dog_breed_cnn_model_V2.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [4]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np

# Load the trained model
loaded_model = load_model(r'\Color\dog_breed_cnn_model_V2.h5')

# Function to preprocess an image for prediction
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalize the image
    return img_array


In [6]:
_# Test the model on a single image
# test_image_path = 'MyDataset/val/n02110185-Siberian_husky/n02110185_931.jpg'  # Change this to the path of your test image
test_image_path = r"\golden_retriever\n02099601_10.jpg"
preprocessed_image = preprocess_image(test_image_path)

# Make a prediction
predictions = loaded_model.predict(preprocessed_image)

# Decode the prediction to get the predicted class
predicted_class = np.argmax(predictions)
print(f"Predicted class index: {predicted_class}")

# Map the predicted class index to the actual class label
class_labels = ['chihuahua','miniature_schnauzer','golden_retriever','rottweiler','great_dane','siberian_husky','pug','pembroke','other']
predicted_label = class_labels[predicted_class]
print(f"Predicted class label: {predicted_label}")

1/1 [==============================] - 2s 2s/step
Predicted class index: 2
Predicted class label: golden_retriever
